In [7]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('241204_MoviesLens')
spark = SparkContext(conf = conf).getOrCreate()

In [8]:
spark

<SparkContext master=local appName=241204_MoviesLens>

### u.data 데이터 설명
user_id : 사용자의 고유 ID  
movie_id : 영화 고유 ID  
rating : 사용자가 영화에 준 평점  
timestamp : 평점이 기록된 시간

In [9]:
import os
directory = os.path.join(os.getcwd(), 'data')
filename = 'u.data'
filepath = os.path.join(directory, filename)
filepath

'/home/lab17/git/src/data/u.data'

In [21]:
datas = spark.textFile('file:///' + filepath.replace('\\','/'))
datas.take(3)

['196\t242\t3\t881250949', '186\t302\t3\t891717742', '22\t377\t1\t878887116']

In [30]:
rows = datas.filter(lambda x: x)

In [95]:
def parse(row):
    field = row.split('\t')
    user_id = field[0]
    movie_id = field[1]
    rating = field[2]
    timestamp = field[3]
    
    return (user_id, movie_id, rating, timestamp)

def parse_only(row):
    field = row.split('\t')
    rating = field[2]
    
    return rating


rows_parse = rows.map(parse)
rows_parse_only = rows.map(parse_only)

In [35]:
rows_parse.take(3)

[('196', '242', '3', '881250949'),
 ('186', '302', '3', '891717742'),
 ('22', '377', '1', '878887116')]

In [96]:
rows_parse_only.take(3)

['3', '3', '1']

### rating별 평점 개수

#### return이 하나인 함수로 단순하게 구하는 법

In [133]:
import pprint

pprint.pprint(rows_parse_only.sortByKey(lambda x : x).countByValue())

defaultdict(<class 'int'>,
            {'1': 6110,
             '2': 11370,
             '3': 27145,
             '4': 34174,
             '5': 21201})


#### return이 4개인 함수에서 groupBy로 구하는 법

In [83]:
rating_res = rows_parse.map(lambda x : int(x[2])).groupBy(lambda x : x).sortByKey(lambda x : [1,2,3,4,5]).collect()
rating_res

[(1, <pyspark.resultiterable.ResultIterable at 0x7fabb574ac10>),
 (2, <pyspark.resultiterable.ResultIterable at 0x7fabb5749040>),
 (3, <pyspark.resultiterable.ResultIterable at 0x7fabb5749460>),
 (4, <pyspark.resultiterable.ResultIterable at 0x7fabb57498e0>),
 (5, <pyspark.resultiterable.ResultIterable at 0x7fabb57493a0>)]

In [92]:
for i,j in rating_res:
    print(i, len(list(j)))

1 6110
2 11370
3 27145
4 34174
5 21201


### 영화ID별 평가 개수

In [117]:
movieID_res = rows_parse.map(lambda x: x[1]).groupBy(lambda x:x).sortBy(lambda x : x).collect()

for i, j in movieID_res:
    print(i, len(j))

1 452
10 89
100 508
1000 10
1001 17
1002 8
1003 8
1004 9
1005 22
1006 23
1007 47
1008 37
1009 64
101 73
1010 44
1011 93
1012 100
1013 38
1014 98
1015 12
1016 137
1017 50
1018 32
1019 31
102 54
1020 35
1021 38
1022 32
1023 31
1024 15
1025 44
1026 4
1027 3
1028 148
1029 14
103 15
1030 20
1031 7
1032 16
1033 32
1034 27
1035 68
1036 24
1037 24
1038 17
1039 90
104 5
1040 25
1041 62
1042 28
1043 8
1044 40
1045 25
1046 46
1047 134
1048 73
1049 25
105 74
1050 43
1051 41
1052 25
1053 24
1054 23
1055 10
1056 10
1057 22
1058 15
1059 35
106 71
1060 39
1061 29
1062 12
1063 41
1064 4
1065 53
1066 16
1067 44
1068 12
1069 18
107 42
1070 27
1071 16
1072 7
1073 66
1074 77
1075 7
1076 12
1077 8
1078 22
1079 45
108 65
1080 2
1081 8
1082 8
1083 6
1084 21
1085 11
1086 21
1087 10
1088 13
1089 38
109 130
1090 37
1091 43
1092 12
1093 23
1094 12
1095 29
1096 3
1097 18
1098 42
1099 13
11 236
110 31
1100 8
1101 74
1102 7
1103 19
1104 4
1105 18
1106 4
1107 18
1108 5
1109 28
111 272
1110 41
1111 7
1112 15
1113 22
1

In [132]:
userID_res = rows_parse.map(lambda x : x[0]).groupBy(lambda x: x).sortBy(lambda x : x).collect()

for i in userID_res:
    print(f' USER_ID {i[0]}가 남긴 댓글 수 : {len(list(i[1]))}')

 USER_ID 1가 남긴 댓글 수 : 272
 USER_ID 10가 남긴 댓글 수 : 184
 USER_ID 100가 남긴 댓글 수 : 59
 USER_ID 101가 남긴 댓글 수 : 67
 USER_ID 102가 남긴 댓글 수 : 216
 USER_ID 103가 남긴 댓글 수 : 29
 USER_ID 104가 남긴 댓글 수 : 111
 USER_ID 105가 남긴 댓글 수 : 23
 USER_ID 106가 남긴 댓글 수 : 64
 USER_ID 107가 남긴 댓글 수 : 22
 USER_ID 108가 남긴 댓글 수 : 33
 USER_ID 109가 남긴 댓글 수 : 234
 USER_ID 11가 남긴 댓글 수 : 181
 USER_ID 110가 남긴 댓글 수 : 133
 USER_ID 111가 남긴 댓글 수 : 24
 USER_ID 112가 남긴 댓글 수 : 46
 USER_ID 113가 남긴 댓글 수 : 51
 USER_ID 114가 남긴 댓글 수 : 48
 USER_ID 115가 남긴 댓글 수 : 92
 USER_ID 116가 남긴 댓글 수 : 143
 USER_ID 117가 남긴 댓글 수 : 86
 USER_ID 118가 남긴 댓글 수 : 71
 USER_ID 119가 남긴 댓글 수 : 181
 USER_ID 12가 남긴 댓글 수 : 51
 USER_ID 120가 남긴 댓글 수 : 26
 USER_ID 121가 남긴 댓글 수 : 74
 USER_ID 122가 남긴 댓글 수 : 61
 USER_ID 123가 남긴 댓글 수 : 54
 USER_ID 124가 남긴 댓글 수 : 24
 USER_ID 125가 남긴 댓글 수 : 182
 USER_ID 126가 남긴 댓글 수 : 45
 USER_ID 127가 남긴 댓글 수 : 23
 USER_ID 128가 남긴 댓글 수 : 184
 USER_ID 129가 남긴 댓글 수 : 30
 USER_ID 13가 남긴 댓글 수 : 636
 USER_ID 130가 남긴 댓글 수 : 353
 USER_ID 131가 남긴 댓글 수

In [34]:
rows_parse.countByValue()

defaultdict(int,
            {('196', '242', '3', '881250949'): 1,
             ('186', '302', '3', '891717742'): 1,
             ('22', '377', '1', '878887116'): 1,
             ('244', '51', '2', '880606923'): 1,
             ('166', '346', '1', '886397596'): 1,
             ('298', '474', '4', '884182806'): 1,
             ('115', '265', '2', '881171488'): 1,
             ('253', '465', '5', '891628467'): 1,
             ('305', '451', '3', '886324817'): 1,
             ('6', '86', '3', '883603013'): 1,
             ('62', '257', '2', '879372434'): 1,
             ('286', '1014', '5', '879781125'): 1,
             ('200', '222', '5', '876042340'): 1,
             ('210', '40', '3', '891035994'): 1,
             ('224', '29', '3', '888104457'): 1,
             ('303', '785', '3', '879485318'): 1,
             ('122', '387', '5', '879270459'): 1,
             ('194', '274', '2', '879539794'): 1,
             ('291', '1042', '4', '874834944'): 1,
             ('234', '1184', '2', '8920